In [1]:
# get country
# get zip and city
# then get state and geolocation

In [2]:
import pandas as pd
import re
from uszipcode import ZipcodeSearchEngine # main
# http://pythonhosted.org/uszipcode/
import zipcode # sub
# https://pypi.python.org/pypi/zipcode
data = pd.read_csv("article_after_processing7.csv", encoding='iso-8859-1')

In [3]:
# get the countries with the top 30 highest GDP
country = pd.read_csv("countries_w_gdp.csv", encoding='iso-8859-1')
country_list = country.Country[0:100]

In [4]:
def get_zipcode(address):
    regex = re.compile(r'\s(\d{5})\s*')
    match = re.search(regex, address)
    if match:
        return match.group(1)
    else:
        return "not found"
    
data["zip_code"] = data.Location.apply(lambda address: get_zipcode(str(address)))

In [5]:
def get_city(zip_code):
    search = ZipcodeSearchEngine()
    if zip_code != "not found":
        myzip = search.by_zipcode(zip_code)
        if myzip:
            return myzip["City"]
        else:
            myzip = zipcode.isequal(zip_code)
            if myzip:
                return myzip.city
        
data["City"] = data.zip_code.apply(lambda zip_code: get_city(zip_code))

In [6]:
def check_city_match(row):
    if row["City"] != None:
        regex = re.compile(r'{}'.format(row["City"]), flags = re.I)
        match = re.search(regex, row["Location"])
        if match:
            return True
        else:
            return False
    else:
        return False


data["address_check"] = data.apply(lambda row: check_city_match(row), axis = 1)

In [7]:
data[ data["address_check"] == False][["Location","zip_code" ,"City"]]

,Location,zip_code,City
0,"Gokaldas Chambers, 3rd Floor, 5th Main, Bellar...",56008,None
1,"6th Floor, New Penderel House 283-288 High Hol...",not found,None
3,"6 Ramillies Street London, W1F 7TY United Kingdom",not found,None
5,"Sonnenburger Str. 73, 10437 Berlin, Germany",10437,None
9,"47 E. All Saints Frederick, MD United States",not found,None
10,"Sovereign House Church Street Brighton, East S...",not found,None
14,The River Building - Level 1 Cannon Bridge Hou...,not found,None
16,BankBazaar.com (A & A Dukaan Financial Service...,60000,None
20,"Max-Beer-Strasse, 2 Berlin, Berlin 10119 Germany",10119,New York
23,"PrannerstraÃ?e 2-4 MÃ¼nchen, 80333 Germany",80333,None


In [8]:
def get_country(row):
    regex = re.compile("|".join(country_list), flags = re.I)
    # address_check == True means city is cross-checked. 
    # This is only possible for US cities
    if row["address_check"]:
        return "United States"
    else:
        if not isinstance(row["Location"], float):
            match = re.search(regex, row["Location"])
            if match:
                return match.group(0)
            else:
                return "not found"
        else:
            return "not found"

data["Country"] = data.apply(lambda location: get_country(location), axis = 1)

In [9]:
data[data["Country"] == "not found"][["Company","Location","zip_code" ,"City"]]

,Company,Location,zip_code,City
79,Ridibooks,"Urban Bench 10th, Teheranro 325 Gangnamgu Seou...",not found,None
215,Cazena,"Waltham, MA",not found,None
227,Narvar,"San Francisco, CA",not found,None


In [10]:
# input country names
def input_countries(row):
    if row.Country == "not found":
        if row.Company == "Ridibooks":
            return "Korea"
        elif row.Company == "Cazena":
            return "United States"
        elif row.Company == "Narvar":
            return "United States"
    else:
        return row.Country
    
data.Country = data.apply(lambda row: input_countries(row), axis = 1)

In [11]:
# confirm the change
data[data["Country"] == "not found"][["Company","Location","zip_code" ,"City"]]

,Company,Location,zip_code,City


In [12]:
mask = (data["Country"] == "United States") & (data["address_check"] == False)
data.loc[mask][["Location","zip_code" ,"City"]].shape

(18, 3)

In [13]:
data.loc[mask][["CompanyName","Location","zip_code" ,"City", "Website"]]

,CompanyName,Location,zip_code,City,Website
9,"Fugue, Inc.","47 E. All Saints Frederick, MD United States",not found,None,http://www.fugue.co
27,Magic Leap,"7500 W Sunrise Blvd Plantation, Florida 33322 ...",33322,Fort Lauderdale,https://www.magicleap.com/#/join
44,2U,United States,not found,None,http://2u.com
51,"Snapchat, Inc.","Ocean Front Walk Venice, CA United States",not found,None,http://www.snapchat.com
59,Oracle | Datalogix,"10075 Westmoor Drive Suite 200 Westminster, CO...",80021,Broomfield,http://www.datalogix.com
60,The Honest Company,"12130 Millennium Drive Los Angeles, CA 90094 U...",90094,Playa Vista,http://www.honest.com
80,Sift Science,". San Francisco, CA United States",not found,None,http://www.siftscience.com
89,Silent Circle,United States,not found,None,https://www.silentcircle.com/
125,"PAX Labs, Inc.","Mission San Francisco, CA United States",not found,None,http://www.pax.com/
131,SentinelOne,"2513 E. Charleston Rd Palo Alto, California 94...",94043,Mountain View,http://www.sentinelone.com


In [14]:
mask = (data["Country"] == "United States") & (data["address_check"] == False) & (data["zip_code"] == "not found")
data.loc[mask][["CompanyName","Location","zip_code" ,"City", "Website"]]

,CompanyName,Location,zip_code,City,Website
9,"Fugue, Inc.","47 E. All Saints Frederick, MD United States",not found,None,http://www.fugue.co
44,2U,United States,not found,None,http://2u.com
51,"Snapchat, Inc.","Ocean Front Walk Venice, CA United States",not found,None,http://www.snapchat.com
80,Sift Science,". San Francisco, CA United States",not found,None,http://www.siftscience.com
89,Silent Circle,United States,not found,None,https://www.silentcircle.com/
125,"PAX Labs, Inc.","Mission San Francisco, CA United States",not found,None,http://www.pax.com/
153,CoreOS,"San Francisco, CA United States",not found,None,https://coreos.com
185,Ticketfly,"San Francisco CA, California United States",not found,None,http://www.ticketfly.com
189,Coupa Software,"San Mateo, CA, United States",not found,None,http://www.coupa.com
197,Formlabs,United States,not found,None,http://formlabs.com


In [15]:
# from google search
from get_company_address import *

# from company website
from get_location_from_company_website import *

# from bloomberg profile
from get_company_address_from_bloomberg import *

def rewrite_location(row):
    if (row["Country"] == "United States") & (row["address_check"] == False) & (row["zip_code"] == "not found"):
        result = get_company_address(row.CompanyName)
        
        if result != "not found":
            print ("done")
            return result
        else:
            result = get_location_trials(row.Website)
            
            if result != "not found":
                print ("done")
                return result
            else:
                result = get_address_from_bloomberg_merged(row.CompanyName)
                if result:
                    print ("done")
                    return result
                else:
                    print ("done")
                    return row.Location
                       
    else:
        print ("pass")
        return row.Location
    
data.Location = data.apply(lambda row: rewrite_location(row), axis = 1)

pass
pass
pass
pass
pass
pass
pass
pass
pass
47 E All Saints St, Frederick, MD 21701
done
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
60 Chelsea Piers #6020, New York, NY 10011
done
pass
pass
pass
pass
pass
pass
not found
not found
move onto to second trial
not found
move onto to third trial
not found
https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=224055283
63 Market Street Venice, CA 90291 United States
done
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
123 Mission St #2000, San Francisco, CA 94105
done
pass
pass
pass
pass
pass
pass
pass
pass
not found
Fairfax, VA 22033
done
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
p

In [16]:
mask = (data["Country"] == "United States") & (data["address_check"] == False) & (data["zip_code"] == "not found")
data.loc[mask][["CompanyName","Location","zip_code" ,"City", "Website"]]

,CompanyName,Location,zip_code,City,Website
9,"Fugue, Inc.","47 E All Saints St, Frederick, MD 21701",not found,None,http://www.fugue.co
44,2U,"60 Chelsea Piers #6020, New York, NY 10011",not found,None,http://2u.com
51,"Snapchat, Inc.","63 Market Street Venice, CA 90291 United States",not found,None,http://www.snapchat.com
80,Sift Science,"123 Mission St #2000, San Francisco, CA 94105",not found,None,http://www.siftscience.com
89,Silent Circle,"Fairfax, VA 22033",not found,None,https://www.silentcircle.com/
125,"PAX Labs, Inc.","660 Alabama St, San Francisco, CA 94110",not found,None,http://www.pax.com/
153,CoreOS,"101 New Montgomery St, San Francisco, CA 94105",not found,None,https://coreos.com
185,Ticketfly,"111 Townsend St, San Francisco, CA 94107",not found,None,http://www.ticketfly.com
189,Coupa Software,"1855 S Grant St, San Mateo, CA 94402",not found,None,http://www.coupa.com
197,Formlabs,"35 Medford St #201, Somerville, MA 02143",not found,None,http://formlabs.com


In [17]:
# assign zip code, city to the new addresses
# conduct address_check

data["zip_code"] = data.Location.apply(lambda address: get_zipcode(str(address)))

data["City"] = data.zip_code.apply(lambda zip_code: get_city(zip_code))

data["address_check"] = data.apply(lambda row: check_city_match(row), axis = 1)

In [18]:
# confirm the change
mask = (data["Country"] == "United States") & (data["address_check"] == False)
data.loc[mask][["CompanyName","Location","zip_code" ,"City", "Website"]]

,CompanyName,Location,zip_code,City,Website
27,Magic Leap,"7500 W Sunrise Blvd Plantation, Florida 33322 ...",33322,Fort Lauderdale,https://www.magicleap.com/#/join
59,Oracle | Datalogix,"10075 Westmoor Drive Suite 200 Westminster, CO...",80021,Broomfield,http://www.datalogix.com
60,The Honest Company,"12130 Millennium Drive Los Angeles, CA 90094 U...",90094,Playa Vista,http://www.honest.com
131,SentinelOne,"2513 E. Charleston Rd Palo Alto, California 94...",94043,Mountain View,http://www.sentinelone.com
134,Varsity Tutors,"101 S. Hanley Rd Suite 300 St. Louis, MO 63105...",63105,Saint Louis,http://www.varsitytutors.com


In [20]:
def get_state(zip_code):
    search = ZipcodeSearchEngine()
    if zip_code != "not found":
        myzip = search.by_zipcode(zip_code)
        if myzip:
            return myzip["State"]
        else:
            myzip = zipcode.isequal(zip_code)
            if myzip:
                return myzip.state
            else:
                return None
    else:
        return None
        
data["State"] = data.zip_code.apply(lambda zip_code: get_state(zip_code))

In [21]:
def get_latitude(zip_code):
    search = ZipcodeSearchEngine()
    if zip_code != "not found":
        myzip = search.by_zipcode(zip_code)
#         myzip = None
        if myzip:
            return myzip["Latitude"]
        else:
            myzip = zipcode.isequal(zip_code)
            if myzip:
                return myzip.lat
            else:
                return None
    else:
        return None
    
data["latitude"] = data.zip_code.apply(lambda zip_code: get_latitude(zip_code))

In [22]:
def get_longitude(zip_code):
    search = ZipcodeSearchEngine()
    if zip_code != "not found":
        myzip = search.by_zipcode(zip_code)
#         myzip = None
        if myzip:
            return myzip["Longitude"]
        else:
            myzip = zipcode.isequal(zip_code)
            if myzip:
                return myzip.lon
            else:
                return None
    else:
        return None
    
data["longitude"] = data.zip_code.apply(lambda zip_code: get_longitude(zip_code))

In [23]:
data.apply(lambda row: sum(row.isnull()))

title                   0
link                    0
excerpt                 0
published_at            0
funding_round          12
money_raised            0
Company                 0
money_raised_float      0
linkedin_link           0
Company_at_Linkedin     0
CompanyName             0
Specialties            33
Industry                0
Website                 0
Location                0
CompanySize             0
Description             0
Founded                 5
Also-viewed             0
zip_code                0
City                   46
address_check           0
Country                 0
State                  46
latitude               46
longitude              46
dtype: int64

In [24]:
data[data["State"].isnull() ]

,title,link,excerpt,published_at,funding_round,money_raised,Company,money_raised_float,linkedin_link,Company_at_Linkedin,...,Description,Founded,Also-viewed,zip_code,City,address_check,Country,State,latitude,longitude
0,Indian online lending platform Capital Float r...,https://techcrunch.com/2017/08/21/indian-onlin...,"Capital Float, which claims to be the largest ...",8/21/2017,Series C,$45 M,Capital Float,45.0,https://www.linkedin.com/company/capital-float,capital float,...,Capital Float is a digital finance company ser...,2013.0,{'Lendingkart': 'https://www.linkedin.com/comp...,56008,None,False,India,None,NaN,NaN
1,"Carwow, a UK startup that helps you buy a new ...",https://techcrunch.com/2017/07/30/carwow-serie...,"Carwow, a platform that helps you buy a new ca...",7/30/2017,Series C,$39 M,Carwow,39.0,https://www.linkedin.com/company/carwow,carwow,...,"carwow is the most convenient, stress-free way...",2013.0,{'carwow GmbH': 'https://www.linkedin.com/comp...,not found,None,False,United Kingdom,None,NaN,NaN
3,"Blowing up the re-location industry, Move Guid...",https://techcrunch.com/2017/07/11/blowing-up-t...,Whatever any country leader says about banning...,7/11/2017,Series C,$48 M,Move Guides,48.0,https://www.linkedin.com/company/move-guides,move guides,...,MOVE Guides helps HR teams move their employee...,2011.0,{'Workable': 'https://www.linkedin.com/company...,not found,None,False,United Kingdom,None,NaN,NaN
5,"After bump in the road, Movinga raises ?17M Se...",https://techcrunch.com/2016/12/08/on-the-movin...,Berlin's startup scene is probably the most go...,12/8/2016,Series C,$20.2 M,Movinga,20.2,https://www.linkedin.com/company/movinga-gmbh,movinga gmbh,...,Movinga is Europeâ??s fastest growing online p...,2015.0,{'Move24 Group GmbH': 'https://www.linkedin.co...,10437,None,False,Germany,None,NaN,NaN
10,Social Intelligence Startup Brandwatch Raises ...,https://techcrunch.com/2015/10/29/social-intel...,"Social media is officially eating Internet, an...",10/29/2015,Series C,$33 M,Brandwatch,33.0,https://www.linkedin.com/company/brandwatch,brandwatch,...,Brandwatch is the worldâ??s leading enterprise...,2007.0,{'Synthesio': 'https://www.linkedin.com/compan...,not found,None,False,United Kingdom,None,NaN,NaN
14,Restaurant Delivery Startup Deliveroo Raises $...,https://techcrunch.com/2015/07/27/series-c-del...,"Deliveroo, the on-demand startup that offers f...",7/27/2015,Series C,$70 M,Deliveroo,70.0,https://www.linkedin.com/company/deliveroo,deliveroo,...,Deliveroo is on a mission to transform the way...,2012.0,{'foodora': 'https://www.linkedin.com/company/...,not found,None,False,United Kingdom,None,NaN,NaN
16,India?s BankBazaar Raises $60M Series C Led By...,https://techcrunch.com/2015/07/02/bankbazaar/,"BankBazaar, a startup that lets Indian consume...",7/2/2015,Series C,$60 M,India?s BankBazaar,60.0,https://www.linkedin.com/company/bank_bazaar,bank_bazaar,...,BankBazaar is the worldâ??s first neutral onli...,2008.0,{'PolicyBazaar.com': 'https://www.linkedin.com...,60000,None,False,India,None,NaN,NaN
23,eGym raises $45M Series C for cloud-connected ...,https://techcrunch.com/2016/03/21/egym/,"eGym, the Munich-based startup that offers clo...",3/21/2016,Series C,$45 M,eGym,45.0,https://www.linkedin.com/company/egym-gmbh,egym gmbh,...,eGym develops professional fitness equipment f...,2010.0,{'eGym Benelux': 'https://www.linkedin.com/com...,80333,None,False,Germany,None,NaN,NaN
28,Lending Platform China Rapid Finance Raises $3...,https://techcrunch.com/2015/07/29/china-rapid-...,"China Rapid Finance, which claims to be the co...",7/29/2015,Series C,$35 M,China Rapid Finance,35.0,https://www.linkedin.com/company/china-rapid-f...,china rapid finance,...,China Rapid Finance (NYSE:XRF) operates one of...,2001.0,{'é??é??æ??': 'https://www.linkedin.com/compan...,20033,None,False,China,None,NaN,NaN
42,Ostrovok Raises New $12M Series C Round To Exp...,https://techcrunch.com/2014/06/18/ostrovok-rai...,"Ostrovok.ru, the Russian acc

In [25]:
data.to_csv("article_after_processing8.csv", index = False)